In [1]:
import pandas as pd
import tensorflow as tf

SHUFFLE_BUFFER = 500
BATCH_SIZE = 2

2024-06-10 22:33:47.060646: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-10 22:33:47.060810: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-10 22:33:47.232718: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
csv_file = tf.keras.utils.get_file('heart.csv', 'https://storage.googleapis.com/download.tensorflow.org/data/heart.csv')

13273/13273 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [3]:
df = pd.read_csv(csv_file)

In [4]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0,fixed,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,normal,1
2,67,1,4,120,229,0,2,129,1,2.6,2,2,reversible,0
3,37,1,3,130,250,0,0,187,0,3.5,3,0,normal,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,normal,0


In [5]:
df.dtypes

age           int64
sex           int64
cp            int64
trestbps      int64
chol          int64
fbs           int64
restecg       int64
thalach       int64
exang         int64
oldpeak     float64
slope         int64
ca            int64
thal         object
target        int64
dtype: object

In [6]:
target = df.pop('target')

https://www.tensorflow.org/tutorials/load_data/pandas_dataframe#a_dataframe_as_an_array

In [7]:
numeric_feature_names = ['age', 'thalach', 'trestbps',  'chol', 'oldpeak']
numeric_features = df[numeric_feature_names]
numeric_features.head()

,age,thalach,trestbps,chol,oldpeak
0,63,150,145,233,2.3
1,67,108,160,286,1.5
2,67,129,120,229,2.6
3,37,187,130,250,3.5
4,41,172,130,204,1.4


In [8]:
tf.convert_to_tensor(numeric_features)

<tf.Tensor: shape=(303, 5), dtype=float64, numpy=
array([[ 63. , 150. , 145. , 233. ,   2.3],
       [ 67. , 108. , 160. , 286. ,   1.5],
       [ 67. , 129. , 120. , 229. ,   2.6],
       ...,
       [ 65. , 127. , 135. , 254. ,   2.8],
       [ 48. , 150. , 130. , 256. ,   0. ],
       [ 63. , 154. , 150. , 407. ,   4. ]])>

This error is odd.  It appears to be independent of the parameter passed, as adapt() takes only an array. And it claims that the value, which is assigned on the previous line, has not been assigned.

In [15]:
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(numeric_features)

UnboundLocalError: local variable 'input_shape' referenced before assignment

In [14]:
help(normalizer.adapt)

Help on method adapt in module keras.src.layers.preprocessing.normalization:

adapt(data) method of keras.src.layers.preprocessing.normalization.Normalization instance
    Computes the mean and variance of values in a dataset.
    
    Calling `adapt()` on a `Normalization` layer is an alternative to
    passing in `mean` and `variance` arguments during layer construction. A
    `Normalization` layer should always either be adapted over a dataset or
    passed `mean` and `variance`.
    
    During `adapt()`, the layer will compute a `mean` and `variance`
    separately for each position in each axis specified by the `axis`
    argument. To calculate a single `mean` and `variance` over the input
    data, simply pass `axis=None` to the layer.
    
    Arg:
        data: The data to train on. It can be passed either as a
            `tf.data.Dataset`, as a NumPy array, or as a backend-native
            eager tensor.
            If a dataset, *it must be batched*. Keras will assume that